# 回测和Portfolio分析 - Alpha-Hunter

本notebook用于：
1. 加载模型预测结果
2. 计算IC、ICIR等评估指标
3. 构建多空组合并回测
4. 分析收益归因和风险特征
5. 可视化cumulative returns和drawdowns


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob

from src.evaluator import PerformanceEvaluator
from src.utils import analyze_prediction_distribution

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', 20)
print("Libraries loaded!")


## 1. 加载预测结果


In [ ]:
# 加载所有模型的预测结果
results_dir = Path('../results')

models = ['transformer', 'ridge', 'random_forest', 'mlp']
predictions_dict = {}

for model in models:
    model_dir = results_dir / model
    if model_dir.exists():
        pred_files = glob.glob(str(model_dir / 'predictions_*.csv'))
        if pred_files:
            latest = max(pred_files, key=lambda x: Path(x).stat().st_mtime)
            df = pd.read_csv(latest)
            df['date'] = pd.to_datetime(df['date'])
            predictions_dict[model] = df
            print(f"{model}: {len(df)} predictions from {df['date'].min().date()} to {df['date'].max().date()}")

if not predictions_dict:
    print("\\n⚠️ 未找到预测结果！请先运行: python ../train.py --model all")
else:
    print(f"\\n成功加载 {len(predictions_dict)} 个模型的预测结果")


## 2. IC分析


In [ ]:
evaluator = PerformanceEvaluator()

# 计算每个模型的IC
ic_dict = {}
for model, preds in predictions_dict.items():
    ic_series = evaluator.compute_ic(preds, method='spearman')
    ic_dict[model] = ic_series

# 可视化IC对比
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# IC时间序列
for model, ic in ic_dict.items():
    axes[0].plot(ic.index, ic.values, label=model, alpha=0.7, linewidth=2)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title('Information Coefficient Over Time')
axes[0].set_ylabel('IC')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# IC累计值
for model, ic in ic_dict.items():
    cumulative_ic = ic.cumsum()
    axes[1].plot(cumulative_ic.index, cumulative_ic.values, label=model, linewidth=2)
axes[1].set_title('Cumulative IC')
axes[1].set_ylabel('Cumulative IC')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(results_dir / 'ic_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# IC统计
ic_stats = pd.DataFrame({
    model: {
        'Mean': ic.mean(),
        'Std': ic.std(),
        'IR': ic.mean() / ic.std() if ic.std() > 0 else 0,
        'IC>0 Ratio': (ic > 0).mean(),
    }
    for model, ic in ic_dict.items()
}).T
print("\\nIC Statistics:")
ic_stats


## 3. Portfolio回测


In [ ]:
# 计算多空组合收益
portfolio_dict = {}
for model, preds in predictions_dict.items():
    portfolio_df = evaluator.compute_portfolio_returns(
        preds,
        long_pct=0.1,  # 买入前10%
        short_pct=0.1,  # 卖空后10%
        transaction_cost=0.003,  # 30bps交易成本
    )
    portfolio_dict[model] = portfolio_df

# 可视化累计收益
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 多空组合累计收益
for model, portfolio in portfolio_dict.items():
    cum_ret = (1 + portfolio['ls_ret_net']).cumprod()
    axes[0, 0].plot(cum_ret.index, cum_ret.values, label=model, linewidth=2)
axes[0, 0].set_title('Long-Short Portfolio Cumulative Returns (Net of Costs)')
axes[0, 0].set_ylabel('Cumulative Return')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 多头组合累计收益
for model, portfolio in portfolio_dict.items():
    cum_ret = (1 + portfolio['long_ret']).cumprod()
    axes[0, 1].plot(cum_ret.index, cum_ret.values, label=model, linewidth=2, alpha=0.7)
axes[0, 1].set_title('Long-Only Portfolio Cumulative Returns')
axes[0, 1].set_ylabel('Cumulative Return')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 月度收益分布
all_returns = []
for model, portfolio in portfolio_dict.items():
    returns = portfolio['ls_ret_net'].values
    all_returns.extend([(model, r) for r in returns])
returns_df = pd.DataFrame(all_returns, columns=['Model', 'Return'])
sns.violinplot(data=returns_df, x='Model', y='Return', ax=axes[1, 0])
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Monthly Return Distribution')
axes[1, 0].tick_params(axis='x', rotation=45)

# Drawdown
for model, portfolio in portfolio_dict.items():
    cum_ret = (1 + portfolio['ls_ret_net']).cumprod()
    running_max = cum_ret.expanding().max()
    drawdown = (cum_ret - running_max) / running_max
    axes[1, 1].plot(drawdown.index, drawdown.values, label=model, linewidth=2)
axes[1, 1].set_title('Drawdown')
axes[1, 1].set_ylabel('Drawdown')
axes[1, 1].set_xlabel('Date')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].fill_between(drawdown.index, 0, drawdown.values, alpha=0.2)

plt.tight_layout()
plt.savefig(results_dir / 'portfolio_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


## 4. 综合性能指标


In [ ]:
# 计算综合统计
summary_stats = {}
for model in predictions_dict.keys():
    preds = predictions_dict[model]
    portfolio = portfolio_dict[model]
    stats = evaluator.compute_summary_statistics(preds, portfolio)
    summary_stats[model] = stats

# 创建比较表
comparison_df = pd.DataFrame(summary_stats).T
comparison_df = comparison_df[[
    'IC_mean', 'IC_IR', 'IC_positive_ratio',
    'LS_mean_return', 'LS_sharpe', 'LS_max_drawdown',
    'Long_mean_return', 'Avg_turnover'
]]

print("\\n📊 Model Performance Comparison:")
print("="*80)
comparison_df.round(4)


In [ ]:
# 可视化指标对比
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = [('IC_IR', 'ICIR'), ('LS_sharpe', 'Sharpe Ratio'), ('LS_max_drawdown', 'Max Drawdown')]
for i, (col, title) in enumerate(metrics):
    comparison_df[col].plot(kind='bar', ax=axes[i], color='steelblue', alpha=0.7)
    axes[i].set_title(title)
    axes[i].set_ylabel('Value')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3, axis='y')
    if col == 'LS_max_drawdown':
        axes[i].set_ylabel('Drawdown (lower is better)')

plt.tight_layout()
plt.savefig(results_dir / 'metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# 保存比较结果
comparison_df.to_csv(results_dir / 'performance_comparison.csv')
print(f"\\n✅ 结果已保存到: {results_dir / 'performance_comparison.csv'}")


## 5. 分档分析（Decile Analysis）


In [ ]:
# 选择最佳模型进行详细分析
best_model = comparison_df['IC_IR'].idxmax()
print(f"最佳模型（根据ICIR）: {best_model}\\n")

best_preds = predictions_dict[best_model]

# 分档分析：将预测分为10档，看每档的实际收益
best_preds['decile'] = best_preds.groupby('date')['prediction'].transform(
    lambda x: pd.qcut(x, 10, labels=False, duplicates='drop')
)

decile_stats = best_preds.groupby('decile').agg({
    'actual_return': ['mean', 'std', 'count']
}).round(4)

print("Decile Analysis (0=lowest prediction, 9=highest):")
print(decile_stats)

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 平均收益
decile_mean = best_preds.groupby('decile')['actual_return'].mean()
axes[0].bar(decile_mean.index, decile_mean.values, color='steelblue', alpha=0.7)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title(f'{best_model.title()}: Average Return by Prediction Decile')
axes[0].set_xlabel('Decile (0=Low, 9=High)')
axes[0].set_ylabel('Average Return')
axes[0].grid(True, alpha=0.3, axis='y')

# 收益分布箱线图
best_preds.boxplot(column='actual_return', by='decile', ax=axes[1])
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title(f'{best_model.title()}: Return Distribution by Decile')
axes[1].set_xlabel('Decile')
axes[1].set_ylabel('Actual Return')
plt.suptitle('')

plt.tight_layout()
plt.savefig(results_dir / f'{best_model}_decile_analysis.png', dpi=300, bbox_inches='tight')
plt.show()


## 总结

本notebook完成了：
1. ✅ 加载多个模型的预测结果
2. ✅ IC分析和可视化
3. ✅ Portfolio回测（多空组合、多头组合）
4. ✅ 综合性能指标比较
5. ✅ 分档分析（Decile Analysis）

### 关键发现
- 最佳模型（ICIR）：查看上面输出
- 所有结果图表已保存到 `results/` 目录

### 下一步
- 前往 `03_interpretation.ipynb` 进行模型可解释性分析
- 或者调整超参数重新训练模型
